In [1]:
import pandas as pd
import numpy as np

GOLD_PATH = '../dataset_and_results/benchmark_dataset.csv'
# PRED_PATH = '../ontology_experiments/results/final_results/llama_70B_owl.csv'

dict_pol = {
    'POS': 'positive',
    'NEG': 'negative',
    'NEU': 'neutral'
}
columns_incls = ['id', 'raw_text', 'aspect_term', 'sentiment_expression', 'target_entity', 'aspect_category', 'sentiment_polarity', 'opinion_holder_span', 'opinion_holder_entity', 'sentiment_intensity', 'opinion_qualifier', 'opinion_reason']

In [2]:
from opinion_overlap_eval import OpinionEval
import os

In [4]:
pred_model_dir = '../dataset_and_results/llm_prediction_results/'
mods = os.listdir(pred_model_dir)
owl_res = 'onto_prompt'
nat_lang_res = 'nlprompt'
op_eval = OpinionEval(columns_incls)
gold_df = pd.read_csv(GOLD_PATH).replace({'sentiment_polarity': dict_pol})
fine_res_rows = []
for mod in mods:
    if "." not in mod:
        mod_dir_path = os.path.join(pred_model_dir, mod)
        
        owl_res_path = os.path.join(mod_dir_path, owl_res, 'csv')
        nat_lang_path = os.path.join(mod_dir_path, nat_lang_res, 'csv')
        print(owl_res_path)
        for owl_file in os.listdir(owl_res_path):
            pred_df = pd.read_csv(os.path.join(owl_res_path, owl_file))
            scores = op_eval.eval_opinions(gold_df, pred_df)
            fine_res_rows.append([mod, 'onto_prompt', owl_file.replace('.csv', ''), round(scores['precision']*100, 2), round(scores['recall']*100, 2), round(scores['f1']*100, 2)])
        for nat_file in os.listdir(nat_lang_path):
            pred_df = pd.read_csv(os.path.join(nat_lang_path, nat_file))
            scores = op_eval.eval_opinions(gold_df, pred_df)
            fine_res_rows.append([mod, 'nlprompt', nat_file.replace('.csv', ''), round(scores['precision']*100, 2), round(scores['recall']*100, 2), round(scores['f1']*100, 2)])



../dataset_and_results/llm_prediction_results/gemma2_9b/onto_prompt/csv
../dataset_and_results/llm_prediction_results/mixtral_8x7b/onto_prompt/csv
../dataset_and_results/llm_prediction_results/gpt-4o/onto_prompt/csv
../dataset_and_results/llm_prediction_results/gpt-4o-mini/onto_prompt/csv
../dataset_and_results/llm_prediction_results/llama_8b/onto_prompt/csv
../dataset_and_results/llm_prediction_results/gemma2_27b/onto_prompt/csv
../dataset_and_results/llm_prediction_results/llama_70b/onto_prompt/csv
../dataset_and_results/llm_prediction_results/mistral_7b/onto_prompt/csv


In [4]:
fine_res_rows

[['gemma2_9b', 'onto_prompt', 'ttl_syntax', 57.44, 58.68, 58.05],
 ['gemma2_9b', 'onto_prompt', 'owl_xml', 56.38, 58.77, 57.55],
 ['gemma2_9b', 'onto_prompt', 'manchester_owl', 56.64, 59.29, 57.94],
 ['gemma2_9b', 'onto_prompt', 'owl_funct', 73.49, 40.12, 51.9],
 ['gemma2_9b', 'onto_prompt', 'rdf_xml', 76.9, 40.79, 53.31],
 ['gemma2_9b', 'onto_prompt', 'obo_format', 57.11, 57.94, 57.52],
 ['gemma2_9b', 'onto_prompt', 'jsonld', 57.92, 58.85, 58.38],
 ['gemma2_9b', 'nlprompt', 'eg_desc_format', 62.29, 61.86, 62.07],
 ['gemma2_9b', 'nlprompt', 'desc_format_eg', 56.24, 61.89, 58.93],
 ['gemma2_9b', 'nlprompt', 'desc_eg_format', 58.1, 62.25, 60.1],
 ['gemma2_9b', 'nlprompt', 'format_eg_desc', 58.62, 57.12, 57.86],
 ['gemma2_9b', 'nlprompt', 'eg_format_desc', 62.3, 61.25, 61.77],
 ['gemma2_9b', 'nlprompt', 'format_desc_eg', 56.51, 60.32, 58.35],
 ['mixtral_8x7b', 'onto_prompt', 'ttl_syntax', 47.79, 62.84, 54.29],
 ['mixtral_8x7b', 'onto_prompt', 'owl_xml', 47.26, 61.13, 53.31],
 ['mixtral_8x

In [5]:
df = pd.DataFrame(fine_res_rows, columns=['model', 'prompt', 'variation',  'precision', 'recall', 'f1'])

In [6]:
df

,model,prompt,variation,precision,recall,f1
0,gemma2_9b,onto_prompt,ttl_syntax,57.44,58.68,58.05
1,gemma2_9b,onto_prompt,owl_xml,56.38,58.77,57.55
2,gemma2_9b,onto_prompt,manchester_owl,56.64,59.29,57.94
3,gemma2_9b,onto_prompt,owl_funct,73.49,40.12,51.90
4,gemma2_9b,onto_prompt,rdf_xml,76.90,40.79,53.31
...,...,...,...,...,...,...
99,mistral_7b,nlprompt,desc_format_eg,46.84,57.77,51.73
100,mistral_7b,nlprompt,desc_eg_format,48.53,54.76,51.45
101,mistral_7b,nlprompt,format_eg_desc,49.64,57.94,53.47
102,mistral_7b,nlprompt,eg_format_desc,46.55,58.02,51.66


In [7]:
onto_res = df[df['prompt'] == 'onto_prompt']
nlp_res = df[df['prompt'] == 'nlprompt']

In [8]:
nlp_res

,model,prompt,variation,precision,recall,f1
7,gemma2_9b,nlprompt,eg_desc_format,62.29,61.86,62.07
8,gemma2_9b,nlprompt,desc_format_eg,56.24,61.89,58.93
9,gemma2_9b,nlprompt,desc_eg_format,58.10,62.25,60.10
10,gemma2_9b,nlprompt,format_eg_desc,58.62,57.12,57.86
11,gemma2_9b,nlprompt,eg_format_desc,62.30,61.25,61.77
12,gemma2_9b,nlprompt,format_desc_eg,56.51,60.32,58.35
20,mixtral_8x7b,nlprompt,eg_desc_format,48.57,62.84,54.79
21,mixtral_8x7b,nlprompt,desc_format_eg,47.66,61.05,53.53
22,mixtral_8x7b,nlprompt,desc_eg_format,47.57,59.83,53.00
23,mixtral_8x7b,nlprompt,format_eg_desc,48.67,60.79,54.06


In [9]:
best_preds_onto = onto_res[onto_res['variation']=='jsonld'][['model', 'precision', 'recall', 'f1']]
best_preds_nlp = nlp_res[nlp_res['variation']=='eg_desc_format'][['model', 'precision', 'recall', 'f1']]

In [10]:
best_preds_nlp

,model,precision,recall,f1
7,gemma2_9b,62.29,61.86,62.07
20,mixtral_8x7b,48.57,62.84,54.79
33,gpt-4o,57.57,66.72,61.81
46,gpt-4o-mini,54.50,63.41,58.62
59,llama_8b,38.52,59.26,46.69
72,gemma2_27b,55.48,65.20,59.95
98,mistral_7b,46.36,59.30,52.04


In [14]:
sorted_res = best_preds_nlp.merge(best_preds_onto, on="model", how="inner", suffixes=["_nlprompt", "_onto_prompt"])

In [15]:
sorted_res

,model,precision_nlprompt,recall_nlprompt,f1_nlprompt,precision_onto_prompt,recall_onto_prompt,f1_onto_prompt
0,gemma2_9b,62.29,61.86,62.07,57.92,58.85,58.38
1,mixtral_8x7b,48.57,62.84,54.79,48.98,61.88,54.68
2,gpt-4o,57.57,66.72,61.81,56.34,65.17,60.44
3,gpt-4o-mini,54.50,63.41,58.62,54.77,63.21,58.69
4,llama_8b,38.52,59.26,46.69,49.13,58.66,53.47
5,gemma2_27b,55.48,65.20,59.95,57.20,64.53,60.65
6,mistral_7b,46.36,59.30,52.04,48.14,56.27,51.89
